# Green Fluorescent Project HW 4

In [51]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.preprocessing import StandardScaler


In [52]:
x_train_data = pd.read_csv('train_X_new.csv')
y_train_data = pd.read_csv('train_y_new.csv')
x_test_data = pd.read_csv('test_X_new.csv')

x_train_data.head(10)

,Unnamed: 0,ConstructedAASeq_cln,Id
0,0,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,11328
1,1,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,5781
2,2,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,13681
3,3,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,30804
4,4,SKGEELFTGVVPILVELDGDVNGHTFSVSGEGEGDATYGELTLKFI...,30813
5,5,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,5983
6,6,SKGEELFTGVVPILVELDGDVNGHKFSESGEGEGDATYGKLTLKFI...,20374
7,7,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,22332
8,8,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,17800
9,9,SKGEELLTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,22064


In [53]:
y_train_data.head(10)

,Unnamed: 0,Brightness_Class,Id
0,0,0,11328
1,1,0,5781
2,2,0,13681
3,3,0,30804
4,4,0,30813
5,5,1,5983
6,6,0,20374
7,7,1,22332
8,8,1,17800
9,9,0,22064


Okay, now that we have checked what the data looks like, let's take out the initials so we can work with only the data. The ID sequence for both x and y are same, so we can save the ID columna separately and then add them back later. Frist column with just order number 01234 also seems useless.

In [54]:
x = x_train_data[['Id', 'ConstructedAASeq_cln']]
print(x)

          Id                               ConstructedAASeq_cln
0      11328  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
1       5781  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
2      13681  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
3      30804  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
4      30813  SKGEELFTGVVPILVELDGDVNGHTFSVSGEGEGDATYGELTLKFI...
...      ...                                                ...
31024   7024  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
31025  14012  SKGEELFTGVVPTLVELDGDVNGHKFSVSGEGAGDATYSKLTLKFI...
31026   4140  SKGEELFTGVVPVLVELDGDVNGHKFSVSGEGEGDATYGKLTLKLI...
31027  15193  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...
31028  30255  SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...

[31029 rows x 2 columns]


In [55]:
y = y_train_data[y_train_data.columns[1]]
y = y.values.ravel()
print(y)

[0 0 0 ... 0 1 1]


In [56]:
ID_column = x_train_data[x_train_data.columns[2]]

ID_column.head(10)

0    11328
1     5781
2    13681
3    30804
4    30813
5     5983
6    20374
7    22332
8    17800
9    22064
Name: Id, dtype: int64

We still have problem that is unlike other datasets, its that we just have letters for x data. Those letters aren't just regular letters too, each of them are amino acids (its their one letter code).  We are going to use connect each of letters and connect them to a descriptors numbers, in different csv file.

# DPPS Feature generation

I'll make a copy of x just so we can manipulate it for DPPS descriptors

In [57]:
x_dpps = x

So let's look at one of the descriptors csv first and get a sense how we are going to do this

In [58]:
DPPS = pd.read_csv('DPPS.csv', skiprows=2)
print(DPPS)


   AA_3 AA_1    D1    D2    D3    D4     D5    D6    D7    D8    D9   D10
0   Ala    A -1.02 -2.88 -0.56  0.36  -6.15 -1.68  0.04 -2.51 -1.94 -0.01
1   Arg    R  1.99  4.13 -4.41 -1.02   4.78  3.04 -9.06  6.71  4.41  0.07
2   Asn    N -2.19  1.86  0.38 -0.13  -2.30  1.41 -5.71 -1.11  1.73 -0.19
3   Asp    D -6.60  3.32  1.61  0.36  -3.25  1.95 -7.36  0.14  1.24 -0.15
4   Cys    C  0.21  1.12  3.42 -0.68  -2.27 -1.22  3.11 -2.98 -1.70  1.57
5   Gln    Q -0.47  1.16 -0.57  0.69   0.39  1.93 -5.46 -0.84  1.93  0.85
6   Glu    E -5.39  0.65 -0.98  1.39  -0.23  2.51 -6.84 -0.68  1.41  1.28
7   Gly    G -2.86 -5.00 -2.97  0.53 -11.45  1.89 -2.11 -3.99 -2.16 -0.76
8   His    H  0.73  2.68 -0.66 -1.89   1.60  1.13 -1.94 -0.11  0.44  0.15
9   Ile    I  1.91 -3.13  0.01  1.14   2.70 -4.55  8.93  0.18 -1.10 -0.76
10  Leu    L  1.64 -2.57  0.00  1.35   2.62 -2.65  7.72  0.05 -1.03 -1.81
11  Lys    K  2.47  1.54 -4.28 -0.86   2.77  2.06 -6.18  2.05  2.19 -1.65
12  Met    M  1.93 -0.01  1.21  0.99  

Let's leave only the stuff that we want.

In [59]:
#list of descriptor columns to be used

desc_cols = DPPS.columns.to_list()
del desc_cols[:2]
print(desc_cols)

['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10']


In [60]:
#create a dictionary to map amino acids to their descriptors

dct_DPPS = dict(zip(DPPS['AA_1'], DPPS.loc[:, desc_cols].apply(list, axis=1).tolist()))
print (dct_DPPS)

{'A': [-1.02, -2.88, -0.56, 0.36, -6.15, -1.68, 0.04, -2.51, -1.94, -0.01], 'R': [1.99, 4.13, -4.41, -1.02, 4.78, 3.04, -9.06, 6.71, 4.41, 0.07], 'N': [-2.19, 1.86, 0.38, -0.13, -2.3, 1.41, -5.71, -1.11, 1.73, -0.19], 'D': [-6.6, 3.32, 1.61, 0.36, -3.25, 1.95, -7.36, 0.14, 1.24, -0.15], 'C': [0.21, 1.12, 3.42, -0.68, -2.27, -1.22, 3.11, -2.98, -1.7, 1.57], 'Q': [-0.47, 1.16, -0.57, 0.69, 0.39, 1.93, -5.46, -0.84, 1.93, 0.85], 'E': [-5.39, 0.65, -0.98, 1.39, -0.23, 2.51, -6.84, -0.68, 1.41, 1.28], 'G': [-2.86, -5.0, -2.97, 0.53, -11.45, 1.89, -2.11, -3.99, -2.16, -0.76], 'H': [0.73, 2.68, -0.66, -1.89, 1.6, 1.13, -1.94, -0.11, 0.44, 0.15], 'I': [1.91, -3.13, 0.01, 1.14, 2.7, -4.55, 8.93, 0.18, -1.1, -0.76], 'L': [1.64, -2.57, 0.0, 1.35, 2.62, -2.65, 7.72, 0.05, -1.03, -1.81], 'K': [2.47, 1.54, -4.28, -0.86, 2.77, 2.06, -6.18, 2.05, 2.19, -1.65], 'M': [1.93, -0.01, 1.21, 0.99, 2.79, -0.56, 5.33, -0.87, -0.99, -1.09], 'F': [2.68, 0.84, 2.22, 0.71, 5.02, -0.3, 8.6, 1.13, -1.4, -0.28], 'P':

In [61]:
#convert the protein sequences to an array of descriptors based on DPPS dictionary

protein_descriptors = []
for sequence in x_dpps['ConstructedAASeq_cln']:
    sequence_descriptor = [np.array(dct_DPPS[amino_acid]) for amino_acid in sequence]
    protein_descriptors.append(np.hstack(sequence_descriptor).flatten())

x_dpps['DPPS'] = protein_descriptors

In [62]:
x_dpps

,Id,ConstructedAASeq_cln,DPPS
0,11328,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."
1,5781,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."
2,13681,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."
3,30804,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."
4,30813,SKGEELFTGVVPILVELDGDVNGHTFSVSGEGEGDATYGELTLKFI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."
...,...,...,...
31024,7024,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."
31025,14012,SKGEELFTGVVPTLVELDGDVNGHKFSVSGEGAGDATYSKLTLKFI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."
31026,4140,SKGEELFTGVVPVLVELDGDVNGHKFSVSGEGEGDATYGKLTLKLI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."
31027,15193,SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFI...,"[-1.76, -0.19, 1.06, -0.69, -5.72, 0.14, -4.14..."


In [63]:
x_dpps['DPPS'][0].shape

(2370,)

I want to pull out just the descriptor values and work with them.

In [64]:
#x_DPPS = x_dpps['DPPS']
#print(x_DPPS)

# Create a new DataFrame by expanding the lists into columns
x_DPPS = pd.DataFrame(x_dpps['DPPS'].tolist())

# Print the new DataFrame
print(x_DPPS)

       0     1     2     3     4     5     6     7     8     9     ...  2360  \
0     -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
1     -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
2     -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
3     -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
4     -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
...     ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   ...   
31024 -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
31025 -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
31026 -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
31027 -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   
31028 -1.76 -0.19  1.06 -0.69 -5.72  0.14 -4.14 -2.42 -0.13  0.69  ...  2.47   

       2361  2362  2363  2364  2365  23

Now that feature generation is complete, let's train out models and see which one does the best. We will only work with DPPS right now, and work with other descriptors one by one.

I will be using a code from last hw to test out different models on this one.

In [65]:
x_train, x_val, y_train, y_val = train_test_split(x_DPPS, y, test_size=0.3, random_state=14)

In [66]:
print(y_val) #this was for debugging purpose

[1 1 0 ... 1 1 0]


In [67]:
models = [
    ('Logistic Regression', LogisticRegression(max_iter=1000)),
    ('Ridge Regression', Ridge(alpha=10.0)),  # adjust the alpha parameter
    ('LASSO', Lasso(alpha=10.0))  # adjust the alpha parameter
]


In [ ]:
# Train and compare the models
best_model = None
best_f1_score = 0.0
best_accuracy = 0.0

for name, model in models:
    if name != 'Logistic Regression':
        # Normalize the features for linear regression models
        scaler = StandardScaler()
        x_train_scaled = scaler.fit_transform(x_train)
        x_val_scaled = scaler.transform(x_val)
        model.fit(x_train_scaled, y_train)
        y_pred = model.predict(x_val_scaled)
    else:
        model.fit(x_train, y_train)
        y_pred = model.predict(x_val)

    # Round the predicted values to 0 or 1
    y_pred = np.round(y_pred)

    # Calculate F1 Score
    f1 = f1_score(y_val, y_pred)

    # Calculate Accuracy
    accuracy = accuracy_score(y_val, y_pred)

    # Print results
    print(f"Model: {name}")
    print(f"F1 Score: {f1}")
    print(f"Accuracy: {accuracy}")
    print()

    # Update best model based on F1 Score
    if f1 > best_f1_score:
        best_f1_score = f1
        best_model = model
        best_accuracy = accuracy

print(f"Best Model (based on F1 Score): {best_model}")
print(f"Best F1 Score: {best_f1_score}")
print(f"Best Accuracy: {best_accuracy}")

The above cell keeps giving me the error of "Target is multiclass but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted']." Which does not make sense to me because my y is definitely 0 to 1. I tried both data frame and also converting it to numpy array, and same thing happens. If you see in my code, I also round it up right before, and it seems to not take that either, and keeps saying it is multi class, not sure why I am running into this problem which I didn't last hw. Anyway.. I guess I am using logistic regression then.

Now let's do GridSearch and see which C value we should use for our binary classification logistic regression model.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import make_scorer, f1_score, accuracy_score



# Step 1: Split your data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_DPPS, y, test_size=0.35, random_state=14)

# Step 2: Preprocess the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# # Step 3: Address class imbalance using SMOTE
# smote = SMOTE(sampling_strategy='auto', random_state=14)
# x_train_resampled, y_train_resampled = smote.fit_resample(x_train_scaled, y_train)


# Step 4: Define a logistic regression model
logistic_model = LogisticRegression(max_iter=1000)

# Step 5: Define a parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.001, 0.005, 0.006, 0.007, 0.01, 0.015, 0.02,0.03,0.05, 0.075, 0.1, 0.15, 0.2, 1.0],  # Specify a range of C values to test
    'penalty': ['l1', 'l2'],  # Test both L1 and L2 penalties
}

# Step 6: Perform GridSearchCV for hyperparameter tuning
scorer = make_scorer(f1_score)  # Use F1 score as the evaluation metric
grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring=scorer)
#grid_search.fit(x_train_resampled, y_train_resampled) #SMOTE
grid_search.fit(x_train_scaled, y_train) #SMOTE

# Get the best hyperparameters from GridSearchCV
best_C = grid_search.best_params_['C']
best_penalty = grid_search.best_params_['penalty']

# Step 7: Train a logistic regression model with the best hyperparameters
best_logistic_model = LogisticRegression(C=best_C, penalty=best_penalty, max_iter=1000)
#best_logistic_model.fit(x_train_resampled, y_train_resampled) #SMOTE
best_logistic_model.fit(x_train_scaled, y_train) #no SMOTE

# Step 8: Make predictions and evaluate
x_val_selected = x_val  # No feature selection applied to validation data
y_val_pred = best_logistic_model.predict(x_val_scaled)

# Calculate F1 Score and Accuracy
f1 = f1_score(y_val, y_val_pred)
accuracy = accuracy_score(y_val, y_val_pred)

# Print performance metrics
print(f"Best C: {best_C}")
print(f"Best Penalty: {best_penalty}")
print(f"F1 Score on Validation Data: {f1}")
print(f"Accuracy on Validation Data: {accuracy}")

Now that we have found the best logistic regression model to use, let's use it on our DPPS model set again and output the excel file.

In [ ]:
x_test_data = pd.read_csv('test_X_new.csv')

print(x_test_data)

ID_column = x_test_data[x_test_data.columns[1]]

print(ID_column)

# Dictionary for DPPS is already created, we just have to use it on our test data set
x_dpps_test = x_test_data

protein_descriptors = []
for sequence in x_dpps_test['ConstructedAASeq_cln']:
    sequence_descriptor = [np.array(dct_DPPS[amino_acid]) for amino_acid in sequence]
    protein_descriptors.append(np.hstack(sequence_descriptor).flatten())

x_dpps_test['DPPS'] = protein_descriptors

x_test = pd.DataFrame(x_dpps_test['DPPS'].tolist())

# Print the new DataFrame
print(x_test)


Let's output our excel file that was used with DPPS descriptors.

In [ ]:
# best_logistic_model = LogisticRegression(C=0.015, penalty='l2', max_iter=1000)

# Step 2: Preprocess the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# Step 4: Fit the model to logistic regression
best_logistic_model = LogisticRegression(C=best_C, penalty=best_penalty, max_iter=1000)
best_logistic_model.fit(x_train_scaled, y_train) # NO SMOTE

# Preprocess the test data (e.g., scaling) - Use the same preprocessing steps as for the training data
x_test_scaled = scaler.transform(x_test)  # Scale the test data using the same scalers

# Make predictions on the test data
y_test_pred = best_logistic_model.predict(x_test_scaled)

# Create a DataFrame with the predicted labels
ID_df = pd.DataFrame({'Id': ID_column})
result_df = pd.DataFrame({'Brightness_class': y_test_pred})
combined_df = pd.concat([ID_df, result_df], axis=1)

# Save the results to a CSV file (replace 'results.csv' with your desired file name)
combined_df.to_csv('DPPS_results.csv', index=False)


# ST-scale Feature Generation

In [ ]:
x_ST = x

ST = pd.read_csv('ST-scale.csv', skiprows=2)
print(ST)

#list of descriptor columns to be used
desc_cols = ST.columns.to_list()
del desc_cols[:2]
print(desc_cols)

#create a dictionary to map amino acids to their descriptors

dct_ST = dict(zip(ST['AA_1'], ST.loc[:, desc_cols].apply(list, axis=1).tolist()))
print (dct_ST)

#convert the protein sequences to an array of descriptors based on DPPS dictionary

protein_descriptors = []
for sequence in x_ST['ConstructedAASeq_cln']:
    sequence_descriptor = [np.array(dct_ST[amino_acid]) for amino_acid in sequence]
    protein_descriptors.append(np.hstack(sequence_descriptor).flatten())

x_ST['ST'] = protein_descriptors



In [ ]:
x_ST

In [ ]:
x_ST['ST'][0].shape

In [ ]:
#x_DPPS = x_dpps['DPPS']
#print(x_DPPS)

# Create a new DataFrame by expanding the lists into columns
x_ST = pd.DataFrame(x_ST['ST'].tolist())

# Print the new DataFrame
print(x_ST)

Now that we got new x_ST using ST scale desciptors, let's train and find our best logistics model.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import make_scorer, f1_score, accuracy_score



# Step 1: Split your data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_ST, y, test_size=0.35, random_state=14)

# Step 2: Preprocess the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# # Step 3: Address class imbalance using SMOTE
# smote = SMOTE(sampling_strategy='auto', random_state=14)
# x_train_resampled, y_train_resampled = smote.fit_resample(x_train_scaled, y_train)


# Step 4: Define a logistic regression model
logistic_model = LogisticRegression(max_iter=1000)

# Step 5: Define a parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.001, 0.005, 0.006, 0.007, 0.01, 0.015, 0.02,0.03,0.05, 0.075, 0.1, 0.15, 0.2, 1.0],  # Specify a range of C values to test
    'penalty': ['l1', 'l2'],  # Test both L1 and L2 penalties
}

# Step 6: Perform GridSearchCV for hyperparameter tuning
scorer = make_scorer(f1_score)  # Use F1 score as the evaluation metric
grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring=scorer)
#grid_search.fit(x_train_resampled, y_train_resampled) #SMOTE
grid_search.fit(x_train_scaled, y_train) #SMOTE

# Get the best hyperparameters from GridSearchCV
best_C = grid_search.best_params_['C']
best_penalty = grid_search.best_params_['penalty']

# Step 7: Train a logistic regression model with the best hyperparameters
best_logistic_model = LogisticRegression(C=best_C, penalty=best_penalty, max_iter=1000)
#best_logistic_model.fit(x_train_resampled, y_train_resampled) #SMOTE
best_logistic_model.fit(x_train_scaled, y_train) #no SMOTE

# Step 8: Make predictions and evaluate
x_val_selected = x_val  # No feature selection applied to validation data
y_val_pred = best_logistic_model.predict(x_val_scaled)

# Calculate F1 Score and Accuracy
f1 = f1_score(y_val, y_val_pred)
accuracy = accuracy_score(y_val, y_val_pred)

# Print performance metrics
print(f"Best C: {best_C}")
print(f"Best Penalty: {best_penalty}")
print(f"F1 Score on Validation Data: {f1}")
print(f"Accuracy on Validation Data: {accuracy}")

Now that we got the model, let's now convert the x_test data into ST scale descriptor

In [ ]:
x_test_data = pd.read_csv('test_X_new.csv')

print(x_test_data)

ID_column = x_test_data[x_test_data.columns[1]]

print(ID_column)

# Dictionary for DPPS is already created, we just have to use it on our test data set
x_ST_test = x_test_data

protein_descriptors = []
for sequence in x_ST_test['ConstructedAASeq_cln']:
    sequence_descriptor = [np.array(dct_ST[amino_acid]) for amino_acid in sequence]
    protein_descriptors.append(np.hstack(sequence_descriptor).flatten())

x_ST_test['ST'] = protein_descriptors

x_test = pd.DataFrame(x_ST_test['ST'].tolist())

# Print the new DataFrame
print(x_test)



Let's pump out different output file using ST scale descriptors

In [ ]:
# best_logistic_model = LogisticRegression(C=0.15, penalty='l2', max_iter=1000)

# Step 2: Preprocess the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# Step 4: Fit the model to logistic regression
best_logistic_model = LogisticRegression(C=best_C, penalty=best_penalty, max_iter=1000)
best_logistic_model.fit(x_train_scaled, y_train) # NO SMOTE

# Preprocess the test data (e.g., scaling) - Use the same preprocessing steps as for the training data
x_test_scaled = scaler.transform(x_test)  # Scale the test data using the same scalers

# Make predictions on the test data
y_test_pred = best_logistic_model.predict(x_test_scaled)

# Create a DataFrame with the predicted labels
ID_df = pd.DataFrame({'Id': ID_column})
result_df = pd.DataFrame({'Brightness_class': y_test_pred})
combined_df = pd.concat([ID_df, result_df], axis=1)

# Save the results to a CSV file (replace 'results.csv' with your desired file name)
combined_df.to_csv('ST_results.csv', index=False)


# T-Scale Descriptor

Now it is a same process, but for T-scale descriptor. A lot of it is repetition, so I will try to minimize the code cells. I have chosen the 3 descriptor I am going to use based on the research and looking at the descriptor variables.

In [ ]:
x_T = x

T = pd.read_csv('T-scale.csv', skiprows=2)
print(T)

#list of descriptor columns to be used
desc_cols = T.columns.to_list()
del desc_cols[:2]
print(desc_cols)

#create a dictionary to map amino acids to their descriptors

dct_T = dict(zip(T['AA_1'], T.loc[:, desc_cols].apply(list, axis=1).tolist()))
print (dct_T)

#convert the protein sequences to an array of descriptors based on DPPS dictionary

protein_descriptors = []
for sequence in x_T['ConstructedAASeq_cln']:
    sequence_descriptor = [np.array(dct_T[amino_acid]) for amino_acid in sequence]
    protein_descriptors.append(np.hstack(sequence_descriptor).flatten())

x_T['T'] = protein_descriptors

# Create a new DataFrame by expanding the lists into columns
x_T = pd.DataFrame(x_T['T'].tolist())

# Print the new DataFrame
print(x_T)

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import make_scorer, f1_score, accuracy_score



# Step 1: Split your data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_ST, y, test_size=0.35, random_state=14)

# Step 2: Preprocess the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# # Step 3: Address class imbalance using SMOTE
# smote = SMOTE(sampling_strategy='auto', random_state=14)
# x_train_resampled, y_train_resampled = smote.fit_resample(x_train_scaled, y_train)


# Step 4: Define a logistic regression model
logistic_model = LogisticRegression(max_iter=1000)

# Step 5: Define a parameter grid for hyperparameter tuning
param_grid = {
    'C': [0.001, 0.005, 0.006, 0.007, 0.01, 0.015, 0.02,0.03,0.05, 0.075, 0.1, 0.15, 0.2, 1.0],  # Specify a range of C values to test
    'penalty': ['l1', 'l2'],  # Test both L1 and L2 penalties
}

# Step 6: Perform GridSearchCV for hyperparameter tuning
scorer = make_scorer(f1_score)  # Use F1 score as the evaluation metric
grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring=scorer)
#grid_search.fit(x_train_resampled, y_train_resampled) #SMOTE
grid_search.fit(x_train_scaled, y_train) #SMOTE

# Get the best hyperparameters from GridSearchCV
best_C = grid_search.best_params_['C']
best_penalty = grid_search.best_params_['penalty']

# Step 7: Train a logistic regression model with the best hyperparameters
best_logistic_model = LogisticRegression(C=best_C, penalty=best_penalty, max_iter=1000)
#best_logistic_model.fit(x_train_resampled, y_train_resampled) #SMOTE
best_logistic_model.fit(x_train_scaled, y_train) #no SMOTE

# Step 8: Make predictions and evaluate
x_val_selected = x_val  # No feature selection applied to validation data
y_val_pred = best_logistic_model.predict(x_val_scaled)

# Calculate F1 Score and Accuracy
f1 = f1_score(y_val, y_val_pred)
accuracy = accuracy_score(y_val, y_val_pred)

# Print performance metrics
print(f"Best C: {best_C}")
print(f"Best Penalty: {best_penalty}")
print(f"F1 Score on Validation Data: {f1}")
print(f"Accuracy on Validation Data: {accuracy}")

In [ ]:
x_test_data = pd.read_csv('test_X_new.csv')

print(x_test_data)

ID_column = x_test_data[x_test_data.columns[1]]

print(ID_column)

# Dictionary for T is already created, we just have to use it on our test data set
x_T_test = x_test_data

protein_descriptors = []
for sequence in x_T_test['ConstructedAASeq_cln']:
    sequence_descriptor = [np.array(dct_T[amino_acid]) for amino_acid in sequence]
    protein_descriptors.append(np.hstack(sequence_descriptor).flatten())

x_T_test['T'] = protein_descriptors

x_test = pd.DataFrame(x_T_test['T'].tolist())

# Print the new DataFrame
print(x_test)

In [ ]:
# best_logistic_model = LogisticRegression(C=0.015, penalty='l2', max_iter=1000)

# Step 2: Preprocess the data
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_val_scaled = scaler.transform(x_val)

# Step 4: Fit the model to logistic regression
best_logistic_model = LogisticRegression(C=best_C, penalty=best_penalty, max_iter=1000)
best_logistic_model.fit(x_train_scaled, y_train) # NO SMOTE

# Preprocess the test data (e.g., scaling) - Use the same preprocessing steps as for the training data
x_test_scaled = scaler.transform(x_test)  # Scale the test data using the same scalers

# Make predictions on the test data
y_test_pred = best_logistic_model.predict(x_test_scaled)

# Create a DataFrame with the predicted labels
ID_df = pd.DataFrame({'ID': ID_column})
result_df = pd.DataFrame({'Brightness_class': y_test_pred})
combined_df = pd.concat([ID_df, result_df], axis=1)

# Save the results to a CSV file (replace 'results.csv' with your desired file name)
combined_df.to_csv('T_results.csv', index=False)